# Game Playing 

In [ ]:
from functools import cached_property

: 

In [ ]:
# Abstract base class for a game state
class GameState:
    def __init__(self):
        self.player = "max" 
        # We need to have player
    
    # Result function: return a state after applying action    
    def result(self, action):
        return None
    
    @property
    def is_terminal(self):
        return False
    
    @cached_property
    def utility(self):
        return 0

    # Return all legal moves
    @cached_property
    def actions(self):
        return []
    
    def __repr__(self):
        return "GameState"
    
    def __hash__(self):
        return hash(repr(self))

# Tic-Tac-Toe Game

In [ ]:
class TicTacToeGameState(GameState):
    def __init__(self, board=None, player="max"): # board and player, board: [[_,_,_],[_,_,_],[_,_,_]]
        self.player = player
        if board is None:
            self.board = [[' ' for _ in range(3)] for _ in range(3)]
        else:
            self.board = board

    def result(self, action):
        # create a successor state after applying an action
        new_board = [row[:] for row in self.board]
        x, y = action # x = row, y = col
        new_board[x][y] = 'X' if self.player == "max" else 'O'
        new_player = "min" if self.player == "max" else "max"
        return TicTacToeGameState(new_board, new_player)

    @cached_property
    def utility(self):
        # +1 = "X" wins, -1 = "O" wins, 0 ties

        # check rows
        for row in self.board:
            if row.count('X') == 3:
                return 1
            if row.count('O') == 3:
                return -1
        # check columns
        for col in range(3):
            if all(self.board[row][col] == 'X' for row in range(3)):
                return 1
            if all(self.board[row][col] == 'O' for row in range(3)):
                return -1
        # check diagonals
        if all(self.board[i][i] == 'X' for i in range(3)):
            return 1
        if all(self.board[i][i] == 'O' for i in range(3)):
            return -1
        if all(self.board[i][2-i] == 'X' for i in range(3)):
            return 1
        if all(self.board[i][2-i] == 'O' for i in range(3)):
            return -1
        # check if all cells are filled, i.e. draw
        if all(cell != ' ' for row in self.board for cell in row):
            return 0
        return -999 # non terminal state

    @property
    def is_terminal(self):
        return self.utility in [-1, 0, 1]
    
    @cached_property
    def actions(self):
        # return all empty cell positions
        res = []
        for i in range(3):
            for j in range(3):
                if self.board[i][j] == ' ':
                    res.append((i, j))
        return res

    def __repr__(self):
        border = '+---+---+---+'
        return '\n'.join([border] + 
                         ['| ' + ' | '.join(row) + ' |' + 
                          '\n' + border 
                          for row in self.board])

    def __hash__(self):
        return hash(''.join([cell for row in self.board for cell in row]) + 
                    self.player)

# Minimax Algorithm

In [ ]:
# 3 cases

def minimax(state):
    if state.is_terminal:
        # return the utility 
        return state.utility, None
    if state.player == "max":
        v = float('-inf') # current minimax value
        best_action = None
        for action in state.actions:
            child = state.result(action)
            child_value, _ = minimax(child)
            if child_value > v:
                v = child_value
                best_action = action
        return v, best_action
    else:
        v = float('inf')
        best_action = None
        for action in state.actions:
            child = state.result(action)
            child_value, _ = minimax(child)
            if child_value < v:
                v = child_value
                best_action = action
        return v, best_action

## Test the minimax implementation

In [ ]:
initial_state = TicTacToeGameState(board=None, player="max")
print(initial_state)
v, best_action = minimax(initial_state)
print(f"Best value: {v}, Best action: {best_action}")

In [ ]:
state = TicTacToeGameState(board=[['X','O',' '], ['O','X',' '], [' ',' ',' ']], 
                           player="max")
print(state)
v, best_action = minimax(state)
print(f"Best value: {v}, Best action: {best_action}")

# Interactive Tic-Tac-Toe Game

In [ ]:
initial_state = TicTacToeGameState(board=None, player="min") # Human makes the first move
current_state = initial_state
print(f"Initial state:\n{current_state}")
while not current_state.is_terminal:
    # human player's turn
    if len(current_state.actions) > 1:
        action = input(f"Enter your move as 'row col': ")
        row, col = [int(x) for x in action.split()]
    else:
        action = current_state.actions[0]
        row, col = action
    print(f"Your action = ({row}, {col})")
    if (row, col) in current_state.actions:
        current_state = current_state.result((row, col))
        print(f"Current state after your move:\n{current_state}")
    else:
        print("Invalid move. Try again.")
        continue
    print()
    # AI's turn
    if not current_state.is_terminal:
        _, best_action = minimax(current_state)
        print(f"AI chooses action = {best_action}")
        current_state = current_state.result(best_action)
        print(f"Current state after AI move:\n{current_state}")

if current_state.utility == 1:
    print("AI wins!")
elif current_state.utility == -1:
    print("You win!")
else:
    print("It's a draw!")

# Alpha-Beta Pruning

In [ ]:
def ab_pruning(state, alpha=float('-inf'), beta=float('inf')):
    if state.is_terminal:
        return state.utility, None
    if state.player == "max":
        v = float('-inf')
        best_action = None
        for action in state.actions:
            child = state.result(action)
            child_value, _ = ab_pruning(child, alpha, beta)
            if child_value > v:
                v = child_value
                best_action = action
            alpha = max(alpha, v)
            if beta <= alpha:
                break
        return v, best_action
    else:
        v = float('inf')
        best_action = None
        for action in state.actions:
            child = state.result(action)
            child_value, _ = ab_pruning(child, alpha, beta)
            if child_value < v:
                v = child_value
                best_action = action
            beta = min(beta, v)
            if beta <= alpha:
                break
        return v, best_action

In [ ]:
initial_state = TicTacToeGameState(board=None, player="max")
print(initial_state)
v, best_action = ab_pruning(initial_state)
print(f"Best value: {v}, Best action: {best_action}")

In [ ]:
state = TicTacToeGameState(board=[['X','O',' '], ['O','X',' '], [' ',' ',' ']], 
                           player="max")
print(state)
v, best_action = ab_pruning(state)
print(f"Best value: {v}, Best action: {best_action}")